In [1]:
# Ensure the output directory exists before writing to it.
!mkdir -p output/netcdfs/landsat7

# Exporting to NetCDF format  

> **Description**  
> The code in this notebook subsets a data cube, selects a specific set of variables, and then outputs that data into a netCDF or GeoTIFF file. The goal is to be able to do external analyses of this data using other data analysis tools or GIS tools. The files would be reasonable in size, since we would restrict the region and parameters in the output.

----  

# Boiler Plate, Loading Data

> ### Import the Datacube

In [2]:
import datacube
dc = datacube.Datacube(app = 'my_app', config = '/home/localuser/.datacube.conf')

>### Browse the available Data Cubes on the storage platform    
> You might want to learn more about what data is stored and how it is stored.


In [3]:
list_of_products = dc.list_products()
netCDF_products = list_of_products[list_of_products['format'] == 'NetCDF']
netCDF_products

,name,description,lon,format,product_type,lat,instrument,platform,time,crs,resolution,tile_size,spatial_dimensions
id,,,,,,,,,,,,,
61,alos2_jjfast_scansar_tile_colombia,ALOS2 PALSAR JJFAST tile in DN format processe...,None,NetCDF,tile,None,PALSAR,ALOS_2,None,EPSG:4326,"[-0.0002666666667, 0.0002666666667]","[0.5333333334, 0.5333333334]","(latitude, longitude)"
65,alos2_palsar2_scansar_caqueta,ALOS2 PALSAR JJFAST tile in DN format processe...,None,NetCDF,tile,None,PALSAR_2,ALOS_2,None,EPSG:4326,"[-0.0002666666667, 0.0002666666667]","[0.5333333334, 0.5333333334]","(latitude, longitude)"
50,alos2_palsar_colombia,ALOS2 PALSAR tile in DN format processed for t...,None,NetCDF,gamma0,None,PALSAR,ALOS_2,None,EPSG:4326,"[-0.0002666666667, 0.0002666666667]","[0.5333333334, 0.5333333334]","(latitude, longitude)"
51,alos2_palsar_kenya,ALOS2 PALSAR tile in DN format processed for t...,None,NetCDF,gamma0,None,PALSAR,ALOS_2,None,EPSG:4326,"[-0.0002666666667, 0.0002666666667]","[0.5333333334, 0.5333333334]","(latitude, longitude)"
52,alos2_palsar_vietnam,ALOS2 PALSAR tile in DN format processed for t...,None,NetCDF,gamma0,None,PALSAR,ALOS_2,None,EPSG:4326,"[-0.0002666666667, 0.0002666666667]","[0.5333333334, 0.5333333334]","(latitude, longitude)"
53,alos_palsar_colombia,ALOS PALSAR tile in DN format processed for th...,None,NetCDF,gamma0,None,PALSAR,ALOS,None,EPSG:4326,"[-0.0002666666667, 0.0002666666667]","[0.5333333334, 0.5333333334]","(latitude, longitude)"
54,alos_palsar_kenya,ALOS PALSAR tile in DN format processed for th...,None,NetCDF,gamma0,None,PALSAR,ALOS,None,EPSG:4326,"[-0.0002666666667, 0.0002666666667]","[0.5333333334, 0.5333333334]","(latitude, longitude)"
55,alos_palsar_vietnam,ALOS PALSAR tile in DN format processed for th...,None,NetCDF,gamma0,None,PALSAR,ALOS,None,EPSG:4326,"[-0.0002666666667, 0.0002666666667]","[0.5333333334, 0.5333333334]","(latitude, longitude)"
14,gpm_imerg_gis_daily_global,Global NetCDF GPM IMERG GIS data,None,NetCDF,daily,None,GPM,GPM,None,EPSG:4326,"[-0.1, 0.1]","[90, 180]","(latitude, longitude)"


>### Pick a product  
>Use the platform names from the previous block to select a small Data Cube. The data_access_api utility will give you lat, lon, and time bounds of your Data Cube.   

In [4]:
import utils.data_cube_utilities.data_access_api as dc_api  
api = dc_api.DataAccessApi(config = '/home/localuser/.datacube.conf')

platform = "LANDSAT_7"
product = "ls7_ledaps_vietnam"

# Get Coordinates
coordinates = api.get_full_dataset_extent(platform = platform, product = product)

/home/local/AMA-INC/jcrattz/data_cube_notebooks/utils/data_cube_utilities/data_access_api.py:283: FutureWarning: casting an xarray.Dataset to a boolean will change in xarray v0.11 to only include data variables, not coordinates. Cast the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  if not dataset:


> #### Display Lat-Lon and Time Bounds

In [5]:
latitude_extents = (min(coordinates['latitude'].values),max(coordinates['latitude'].values))
print( latitude_extents )

(9.176425374578418, 13.964805165051667)


In [6]:
longitude_extents = (min(coordinates['longitude'].values),max(coordinates['longitude'].values))
print( longitude_extents )

(102.40430421277932, 108.93092407802477)


In [7]:
time_extents = (min(coordinates['time'].values),max(coordinates['time'].values))
print( time_extents )

(numpy.datetime64('1999-09-08T03:13:19.000000000'), numpy.datetime64('2016-12-29T03:10:00.000000000'))


# Visualize Data Cube Region

In [8]:
## The code below renders a map that can be used to orient yourself with the region.
from utils.data_cube_utilities.dc_display_map import display_map
display_map(latitude = latitude_extents, longitude = longitude_extents)

> #### Picking a smaller analysis region

In [9]:
# ######### Colombia - Cartegena ##################
# longitude_extents = (-74.863, -74.823)
# latitude_extents = (1.326, 1.357)

######### Vietnam - Buan Tua Srah Lake ################## 
longitude_extents = (108.02, 108.15)
latitude_extents  = (12.18 , 12.30)

######### Vietnam - Central Lam Dong Province ################## 
# longitude_extents = (107.8118, 108.0314)
# latitude_extents  = (11.7408, 11.8990)

######## Kenya - Lake Nakuru ##################
# longitude_extents = (36.02, 36.13)
# latitude_extents = (-0.42, -0.28) 

time_extents = ('2015-01-01', '2016-01-01')
print ( time_extents )

('2015-01-01', '2016-01-01')


In [10]:
display_map(latitude = latitude_extents, longitude = longitude_extents)

In [11]:
landsat_dataset = dc.load(latitude = latitude_extents,
                          longitude = longitude_extents,
                          platform = platform,
                          time = time_extents,
                          product = product,
                         measurements = ['red', 'green', 'blue', 'nir', 'swir1', 'swir2', 'pixel_qa']) 

In [12]:
landsat_dataset

<xarray.Dataset>
Dimensions:    (latitude: 446, longitude: 483, time: 19)
Coordinates:
  * time       (time) datetime64[ns] 2015-01-09T03:06:13 2015-01-25T03:06:16 ...
  * latitude   (latitude) float64 12.3 12.3 12.3 12.3 12.3 12.3 12.3 12.3 ...
  * longitude  (longitude) float64 108.0 108.0 108.0 108.0 108.0 108.0 108.0 ...
Data variables:
    red        (time, latitude, longitude) int16 912 1008 1125 989 951 1317 ...
    green      (time, latitude, longitude) int16 674 779 842 863 801 990 969 ...
    blue       (time, latitude, longitude) int16 493 473 574 554 473 714 714 ...
    nir        (time, latitude, longitude) int16 2500 2544 2587 2544 2630 ...
    swir1      (time, latitude, longitude) int16 2740 3002 3054 2871 2662 ...
    swir2      (time, latitude, longitude) int16 1678 1982 2037 1872 1677 ...
    pixel_qa   (time, latitude, longitude) int32 66 66 66 66 66 66 66 66 66 ...
Attributes:
    crs:      EPSG:4326

# Derive Several Products

>### Unpack pixel_qa

In [13]:
import xarray as xr  
import numpy as np

def ls7_unpack_qa( data_array , cover_type):  
    
    land_cover_endcoding = dict( fill     =  [1], 
                                 clear    =  [66,  130], 
                                 water    =  [68,  132],
                                 shadow   =  [72,  136],
                                 snow     =  [80,  112, 144, 176],
                                 cloud    =  [96,  112, 160, 176, 224],
                                 low_conf =  [66,  68,  72,  80,  96,  112],
                                 med_conf =  [130, 132, 136, 144, 160, 176],
                                 high_conf=  [224]
                               ) 
    boolean_mask = np.isin(data_array.values, land_cover_endcoding[cover_type]) 
    return xr.DataArray(boolean_mask.astype(int),
                        coords = data_array.coords,
                        dims = data_array.dims,
                        name = cover_type + "_mask",
                        attrs = data_array.attrs)  

In [14]:
clear_xarray  = ls7_unpack_qa(landsat_dataset.pixel_qa, "clear")  
water_xarray  = ls7_unpack_qa(landsat_dataset.pixel_qa, "water")

shadow_xarray = ls7_unpack_qa(landsat_dataset.pixel_qa, "shadow")  

In [15]:
clean_xarray = xr.ufuncs.logical_or(clear_xarray , water_xarray).astype(np.int8).rename("clean_mask")

clean_mask = np.logical_or(clear_xarray.values.astype(bool),
                           water_xarray.values.astype(bool)) 

> ### Water

In [16]:
from utils.data_cube_utilities.dc_water_classifier import wofs_classify

water_classification = wofs_classify(landsat_dataset,
                                     clean_mask = clean_mask, 
                                     mosaic = False) 

/home/local/AMA-INC/jcrattz/data_cube_notebooks/utils/data_cube_utilities/dc_water_classifier.py:74: RuntimeWarning: divide by zero encountered in true_divide
  return (a - b) / (a + b)


In [17]:
wofs_xarray = water_classification.wofs

> ###  Normalized Indices  

In [18]:
def NDVI(dataset):
    return ((dataset.nir - dataset.red)/(dataset.nir + dataset.red)).rename("NDVI")

In [19]:
def NDWI(dataset):
    return ((dataset.green - dataset.nir)/(dataset.green + dataset.nir)).rename("NDWI")

In [20]:
def NDBI(dataset):
        return ((dataset.swir2 - dataset.nir)/(dataset.swir2 + dataset.nir)).rename("NDBI")

In [21]:
ndbi_xarray = NDBI(landsat_dataset)  # Urbanization - Reds
ndvi_xarray = NDVI(landsat_dataset)  # Dense Vegetation - Greens
ndwi_xarray = NDWI(landsat_dataset)  # High Concentrations of Water - Blues  

>### TSM  

In [22]:
from utils.data_cube_utilities.dc_water_quality import tsm

tsm_xarray = tsm(landsat_dataset, clean_mask = wofs_xarray.values.astype(bool) ).tsm

> ### EVI  

In [23]:
def EVI(dataset, c1 = None, c2 = None, L = None):
        return ((dataset.nir - dataset.red)/((dataset.nir  + (c1 * dataset.red) - (c2 *dataset.blue) + L))).rename("EVI")

In [24]:
evi_xarray = EVI(landsat_dataset, c1 = 6, c2 = 7.5, L = 1 )

# Combine Everything  

In [25]:
combined_dataset = xr.merge([landsat_dataset,
          clean_xarray,
          clear_xarray,
          water_xarray,
          shadow_xarray,
          evi_xarray,
          ndbi_xarray,
          ndvi_xarray,
          ndwi_xarray,
          wofs_xarray,
          tsm_xarray])

# Copy original crs to merged dataset 
combined_dataset = combined_dataset.assign_attrs(landsat_dataset.attrs)

combined_dataset

<xarray.Dataset>
Dimensions:      (latitude: 446, longitude: 483, time: 19)
Coordinates:
  * time         (time) datetime64[ns] 2015-01-09T03:06:13 ...
  * latitude     (latitude) float64 12.3 12.3 12.3 12.3 12.3 12.3 12.3 12.3 ...
  * longitude    (longitude) float64 108.0 108.0 108.0 108.0 108.0 108.0 ...
Data variables:
    red          (time, latitude, longitude) float32 912.0 1008.0 1125.0 ...
    green        (time, latitude, longitude) float32 674.0 779.0 842.0 863.0 ...
    blue         (time, latitude, longitude) float32 493.0 473.0 574.0 554.0 ...
    nir          (time, latitude, longitude) float32 2500.0 2544.0 2587.0 ...
    swir1        (time, latitude, longitude) float32 2740.0 3002.0 3054.0 ...
    swir2        (time, latitude, longitude) float32 1678.0 1982.0 2037.0 ...
    pixel_qa     (time, latitude, longitude) int32 66 66 66 66 66 66 66 66 ...
    clean_mask   (time, latitude, longitude) int8 1 1 1 1 1 1 1 1 1 1 1 1 1 ...
    clear_mask   (time, latitude, longitude

# Export NetCDF

In [26]:
output_file_name  =  "output/netcdfs/landsat7/ls7_netcdf_example.nc"
import os
if os.path.isfile(output_file_name):
    os.remove(output_file_name)
dataset_to_output =  combined_dataset
combined_dataset = combined_dataset.assign_attrs(landsat_dataset.attrs)
datacube.storage.storage.write_dataset_to_netcdf(dataset_to_output, output_file_name)

----  

Sanity check using `gdalinfo` to make sure that all of our bands exist  

In [27]:
!gdalinfo output/netcdfs/landsat7/ls7_netcdf_example.nc

Driver: netCDF/Network Common Data Format
Files: output/netcdfs/landsat7/ls7_netcdf_example.nc
Size is 512, 512
Coordinate System is `'
Metadata:
  NC_GLOBAL#Conventions=CF-1.6, ACDD-1.3
  NC_GLOBAL#date_created=2018-07-02T11:49:04.105173
  NC_GLOBAL#geospatial_bounds=POLYGON ((108.019897632634468 12.300002364756278,108.019897632634468 12.179807779741022,108.150063517303465 12.179807779741022,108.150063517303465 12.300002364756278,108.019897632634468 12.300002364756278))
  NC_GLOBAL#geospatial_bounds_crs=EPSG:4326
  NC_GLOBAL#geospatial_lat_max=12.30000236475628
  NC_GLOBAL#geospatial_lat_min=12.17980777974102
  NC_GLOBAL#geospatial_lat_units=degrees_north
  NC_GLOBAL#geospatial_lon_max=108.1500635173035
  NC_GLOBAL#geospatial_lon_min=108.0198976326345
  NC_GLOBAL#geospatial_lon_units=degrees_east
  NC_GLOBAL#history=NetCDF-CF file created by datacube version '0+untagged.3197.g59d9eb7.dirty' at 20180702.
Subdatasets:
  SUBDATASET_1_NAME=NETCDF:"output/netcdfs/landsat7/ls7_netcdf_exampl